In [17]:
import nltk
import pickle
import argparse
from collections import Counter
import json
import os
from tqdm import *
import numpy as np
import re
import pickle

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word, idx=None):
        if idx is None:
            if not word in self.word2idx:
                self.word2idx[word] = self.idx
                self.idx2word[self.idx] = word
                self.idx += 1
            return self.idx
        else:
            if not word in self.word2idx:
                self.word2idx[word] = idx
                if idx in self.idx2word.keys():
                    self.idx2word[idx].append(word)
                else:
                    self.idx2word[idx] = [word]

                return idx

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<pad>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

In [65]:
def build_vocab_dataset():
    
    #f1 = open(os.getcwd() +"/Annotations/classes.txt", "r")
    #idx = 0
    #class_names = {}
    #for class_name in f1:
    #    class_name = class_name.strip()
    #    class_names[idx] = class_name
    #    idx += 1
    #f1.close()
    
    f = open(os.getcwd() +"/Annotations/ingredients_Recipes5k.txt", "r")
    class_to_ingrd = {}
    counter_ingrs = Counter()
    idx = 0
    max_length= -1
    i = 0
    for line in f:
        items = get_ingredients(line)
        #class_to_ingrd[class_names[idx]] = items
        idx += 1
        if len(items) > max_length:
            max_length = len(items)
        counter_ingrs.update(items)
    print("The maximum length of ingredients: ", max_length)
    f.close()
    
    
    print(counter_ingrs)
    counter_ingrs, cluster_ingrs = cluster_ingredients(counter_ingrs)
    counter_ingrs, cluster_ingrs = remove_plurals(counter_ingrs, cluster_ingrs)
    print(sorted(counter_ingrs.items())
    
    
    vocab_ingrs = Vocabulary()
    idx = vocab_ingrs.add_word('<end>')
    for k, _ in counter_ingrs.items():
        for ingr in cluster_ingrs[k]:
            idx = vocab_ingrs.add_word(ingr, idx)
        idx += 1
    _ = vocab_ingrs.add_word('<pad>', idx)
    
    print("Total ingr vocabulary size: {}".format(len(vocab_ingrs)))
    dataset = {'train': [], 'val': [], 'test': []}
    for split in ['train', 'val', 'test']:
        f2 = open(os.getcwd() +"/Annotations/"+split+'_images.txt', "r")
        temp_name = None
        idx = -1
        for line in f2:
            line = line.strip()
            name, number = line.split('/')
            if name != temp_name:
                idx += 1
                temp_name = name
            #newentry = {'id': temp_name,'ingredients': class_to_ingrd[class_names[idx]], 'images': line}
            newentry = {'id': temp_name, 'images': line}
            dataset[split].append(newentry)
    return vocab_ingrs, dataset

SyntaxError: invalid syntax (<ipython-input-65-d987b710fc86>, line 35)

In [66]:
def cluster_ingredients(counter_ingrs):
    mydict = dict()
    mydict_ingrs = dict()

    for k, v in counter_ingrs.items():

        w1 = k.split('_')[-1]
        w2 = k.split('_')[0]
        lw = [w1, w2]
        if len(k.split('_')) > 1:
            w3 = k.split('_')[0] + '_' + k.split('_')[1]
            w4 = k.split('_')[-2] + '_' + k.split('_')[-1]

            lw = [w1, w2, w4, w3]

        gotit = 0
        for w in lw:
            if w in counter_ingrs.keys():
                # check if its parts are
                parts = w.split('_')
                if len(parts) > 0:
                    if parts[0] in counter_ingrs.keys():
                        w = parts[0]
                    elif parts[1] in counter_ingrs.keys():
                        w = parts[1]
                if w in mydict.keys():
                    mydict[w] += v
                    mydict_ingrs[w].append(k)
                else:
                    mydict[w] = v
                    mydict_ingrs[w] = [k]
                gotit = 1
                break
        if gotit == 0:
            mydict[k] = v
            mydict_ingrs[k] = [k]

    return mydict, mydict_ingrs

def get_ingredients(line):   
    line = line.lower().strip('\n')
    items = line.split(",")
    while("" in items):
        items.remove("")
    #print(items)
    for i in range(len(items)):
        if len(items[i]) > 0:
            items[i] = items[i].replace(' ', '_')
            items[i] = items[i].strip('\n')
        else:
            del items[i]
    return items

def remove_plurals(counter_ingrs, ingr_clusters):
    del_ingrs = []

    for k, v in counter_ingrs.items():

        if len(k) == 0:
            del_ingrs.append(k)
            continue

        gotit = 0
        if k[-2:] == 'es':
            if k[:-2] in counter_ingrs.keys():
                counter_ingrs[k[:-2]] += v
                ingr_clusters[k[:-2]].extend(ingr_clusters[k])
                del_ingrs.append(k)
                gotit = 1

        if k[-1] == 's' and gotit == 0:
            if k[:-1] in counter_ingrs.keys():
                counter_ingrs[k[:-1]] += v
                ingr_clusters[k[:-1]].extend(ingr_clusters[k])
                del_ingrs.append(k)

    for item in del_ingrs:
        del counter_ingrs[item]
        del ingr_clusters[item]

    return counter_ingrs, ingr_clusters

In [67]:
vocab_ingrs, dataset = build_vocab_dataset()

The maximum length of ingredients:  65
Counter({'salt': 2076, 'eggs': 894, 'butter': 791, 'sugar': 773, 'olive_oil': 761, 'garlic': 737, 'water': 643, 'pepper': 637, 'onions': 539, 'all-purpose_flour': 484, 'unsalted_butter': 478, 'kosher_salt': 468, 'milk': 467, 'large_eggs': 453, 'vegetable_oil': 426, 'vanilla_extract': 421, 'soy_sauce': 418, 'flour': 370, 'garlic_cloves': 353, 'baking_powder': 350, 'ground_black_pepper': 314, 'green_onions': 314, 'carrots': 314, 'granulated_sugar': 292, 'heavy_cream': 289, 'lemon_juice': 280, 'extra-virgin_olive_oil': 264, 'sesame_oil': 263, 'black_pepper': 256, 'scallions': 255, 'powdered_sugar': 250, 'avocado': 243, 'oil': 236, 'mayonnaise': 231, 'baking_soda': 230, 'purple_onion': 222, 'sea_salt': 219, 'brown_sugar': 219, 'tomatoes': 214, 'grated_parmesan_cheese': 211, 'ginger': 203, 'cinnamon': 201, 'ground_cinnamon': 200, 'lemon': 200, 'rice_vinegar': 198, 'worcestershire_sauce': 196, 'dijon_mustard': 195, 'vanilla': 194, 'garlic_powder': 192, 

In [61]:
print(vocab_ingrs)

In [58]:
def main():

    vocab_ingrs, dataset = build_vocab_dataset()

    with open(os.path.join(os.getcwd(), 'vocab_ingrs.pkl'), 'wb') as f:
        pickle.dump(vocab_ingrs, f)

    for split in dataset.keys():
        with open(os.path.join(os.getcwd(),'data' + split + '.pkl'), 'wb') as f:
            pickle.dump(dataset[split], f)


In [59]:
main()

The maximum length of ingredients:  65
Total ingr vocabulary size: 978


In [60]:
print(len(dataset['train']), len(dataset['test']),len(dataset['val']))

3409 783 634
